In [1]:
#imports
from data_loader import DataLoader
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

In [2]:
dl = DataLoader("./config.json") # TODO load bank full later
inputs, labels = dl.load_csv_data() #  TODO load all
print("number of null elements:", inputs.isnull().values.sum())
full_df = inputs.join(labels)
full_df.head()

loading data from: /home/duje/Desktop/DOKTORSKI/predmeti-doktorski/otkrivanje_znanja/bank-marketing/data/bank.csv
number of null elements: 0


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


# converting to one-hot

## variables that need to be encoded:
### numerical: None
### categorical: marital, education, default, housing, loan, contact, poutcome
month -> one hot ili samo label encode ?

In [3]:
all_columns_names = inputs.columns

nominal_columns_names = ['job', 'marital', 'education', 'default', 'housing', 'loan', 
                         'contact', 'poutcome'] # <- the target label is also added
nominal_columns = full_df[nominal_columns_names]

ordinal_columns_names = ['month', 'y'] # add education also? How to encode moths?
ordinal_columns = full_df[ordinal_columns_names]

numerical_columns_names = list(set(all_columns_names) - 
                               set(nominal_columns_names) - 
                               set(ordinal_columns_names))
numerical_columns = full_df[numerical_columns_names]

In [7]:
# encode ordinal categorical values (month)
ordinal_columns.month.unique()
mymap = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 
         'sep':9, 'oct':10, 'nov':11, 'dec':12, 'yes':1, 'no': 0}

encoded_ordinal_columns = ordinal_columns.applymap(lambda s: mymap.get(s) if s in mymap else s)
encoded_ordinal_columns.head()

,month,y
0,10,0
1,5,0
2,4,0
3,6,0
4,5,0


In [5]:
# encode nominal categorical values
encoded_nominal_df=None
for column_name in nominal_columns_names:
    oh_columns = pd.get_dummies(nominal_columns[column_name], prefix=column_name)
    if encoded_nominal_df is None:
        encoded_nominal_df = oh_columns
    else:
        encoded_nominal_df = pd.concat([encoded_nominal_df, oh_columns], axis=1)

encoded_nominal_df.head()

,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,1
1,0,0,0,0,0,0,0,1,0,0,...,1,0,1,1,0,0,1,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,1,1,0,1,0,0,1,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,1,0,1,0,0,1,0,0,0,1
4,0,1,0,0,0,0,0,0,0,0,...,1,1,0,0,0,1,0,0,0,1


In [8]:
pd.set_option('display.max_columns', None)
df = pd.concat([numerical_columns, encoded_nominal_df, encoded_ordinal_columns], axis=1)
df.head()

,campaign,day,previous,age,balance,pdays,duration,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,month,y
0,1,19,0,30,1787,-1,79,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,10,0
1,1,11,4,33,4789,339,220,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,1,0,0,1,0,0,0,5,0
2,1,16,1,35,1350,330,185,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,0,1,0,0,0,4,0
3,4,3,0,30,1476,-1,199,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,1,6,0
4,1,5,0,59,0,-1,226,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,1,5,0
